# Import packages and data

In [2]:
import pandas as pd
import numpy as np

In [3]:
POLICY_DATA = '../data/datastorm_policy_data.csv'
AGENT_DATA = '../data/datastorm_agent_data.csv'

In [4]:
df = pd.read_csv(POLICY_DATA)

c:\users\nisal_105136\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,main_holder_occupation_cd,policy_term,policy_payment_mode,policy_status,commencement_dt,next_due_dt,termination_dt,termination_reason,main_holder_gender,main_holder_dob,...,child5_gender,payment_method,main_holder_occupation,policy_snapshot_as_on,run_date,client_code,agent_code,product_code,product_name,policy_code
0,249.0,20,M,INFORCE,2011/08/21,2019/02/21,NaN,NaN,M,1983/10/01,...,NaN,CASH,NaN,20190131,20200701,C108223,AG108847,HE001,HEALTH,POL28535
1,165.0,15,M,INFORCE,2006/07/14,2019/09/14,NaN,NaN,F,1958/01/01,...,NaN,CASH,BUSINESSMAN,20190901,20200701,C103349,AG103677,RE001,RETIREMENT,POL27771
2,330.0,20,Y,INFORCE,2018/12/28,2019/12/28,NaN,NaN,M,1967/11/01,...,NaN,CASH,RECYCLE RACERS,20191101,20200701,C120388,AG103249,IN001,INVESTMENT,POL29786
3,415.0,15,Q,INFORCE,2018/11/06,2020/02/06,NaN,NaN,M,1989/10/01,...,NaN,CHEQUE,RECEPTIONIST,20191201,20200701,C113155,AG106733,HE001,HEALTH,POL24635
4,165.0,15,M,LAPSED,2012/04/28,2015/06/28,NaN,OTHERS,M,1988/05/01,...,NaN,CASH,BUSINESSMAN,20190228,20200701,C106155,AG106907,HE001,HEALTH,POL7470


# Create Labels

In [189]:
def create_labels(df, train_snap_date, label_last_date):
    snap_df = df[(df.policy_snapshot_as_on == train_snap_date) 
                    & (df.policy_status == 'INFORCE')][['client_code', 
                                                        'policy_code', 
                                                        'product_name']]
    
    cust_df = snap_df[['client_code']].drop_duplicates().set_index('client_code')

    label_df = df[(df.policy_snapshot_as_on > train_snap_date) 
                     & (df.policy_snapshot_as_on <= label_last_date) 
                     & (df.policy_status == 'INFORCE')][['client_code', 
                                                         'product_name', 
                                                         'policy_code', 
                                                         'policy_snapshot_as_on']]
    
    join_df = label_df.merge(snap_df[['policy_code', 'client_code']], on=['policy_code', 'client_code'], how='left', indicator=True)
    join_df['is_prev'] = join_df._merge.map({'left_only':0, 'both':1}).astype(int)
    
    #customers who can be c
    cross_sell_eligible = join_df.groupby(['client_code', 'policy_snapshot_as_on']).is_prev.sum()
    
    cross_sell_eligible.name = 'cross_sell_eligible'
    cross_sell_eligible = cross_sell_eligible[cross_sell_eligible > 0].reset_index()
    
    joined_cs_eligible = join_df.merge(cross_sell_eligible, on=['client_code', 'policy_snapshot_as_on'], how='left').dropna()
    
    labels = joined_cs_eligible[joined_cs_eligible.is_prev == 0][['client_code', 'product_name']].drop_duplicates()
    labels['value'] = 1
    pivot_labels = labels.pivot_table(values='value', index='client_code', aggfunc='sum', columns='product_name').fillna(0).astype(int)

    return cust_df.merge(pivot_labels, on='client_code', how='left').fillna(0)